In [1]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
import gradio as gr

# Authenticate client
prediction_key = "07051a00c54f429aacf110c380a4e3fd"
prediction_endpoint = "https://team1thetteamcv-prediction.cognitiveservices.azure.com/"
project_id = "a9a4bcb6-6f63-4990-8811-806c5f3f0ce7"
model_name = "Iteration2"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

data = {
    "Inputs": {
        "input1": [
            {
                "paper": "이건 종이야!📄\n종이는 종이끼리 모여진 데 버려주면 돼!",
                "glass": "이건 유리야!🔍\n유리병은 깨질 수 있으니 던지지 말고 조심히 버려줘!",
                "metal": "이건 금속이야!🔧\n날카로운 모서리에 손이 베이지 않게 조심해!",
                "plastic": "이건 플라스틱이야!🌱\n페트병은 라벨을 떼고 찌그러트려 버려줘!",
                "trash": "이건 일반쓰레기야!🗑️\n일반쓰레기는 종량제 봉투에 모아줄래?\n종량제 봉투가 뭐냐구? 함께 알아볼까?"
            }
        ]
    },
    "GlobalParameters": {}
}

def process_image(image):
    # 이미지를 512x512로 리사이즈
    image = image.resize((512, 512))
    byte_arr = io.BytesIO()

    image.save(byte_arr, format='JPEG')
    return byte_arr.getvalue()

def predict_image(input_img):
    input_img = input_img.resize((512, 512))  # 이미지를 512x512로 리사이즈
    processed_image = process_image(input_img)
    
    results = predictor.classify_image(project_id, model_name, processed_image)  # 이미지 분류용 메서드
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.axis('off')
    
    draw = ImageDraw.Draw(input_img)
    color = 'magenta'
    lineWidth = 5
    box_size = 350
    center_x, center_y = 512 // 2, 512 // 2
    
    detected_objects = []
    
    for prediction in results.predictions:
        print(f"{prediction.tag_name}: {prediction.probability * 100:.2f}%")
        if (prediction.probability * 100) > 70:  # 확률 임계값을 50%로 낮춤
            detected_objects.append(prediction.tag_name)
            
            left = center_x - box_size // 2
            top = center_y - box_size // 2
            right = center_x + box_size // 2
            bottom = center_y + box_size // 2

            draw.rectangle([left, top, right, bottom], outline=color, width=lineWidth)
            draw.text((left + 5, top - 15), f"{prediction.tag_name}: {prediction.probability * 100:.2f}%", fill=color)
    
    ax.imshow(input_img)
    
    # 예측 결과 텍스트 생성
    if detected_objects:
        predictions_text = "\n".join(data["Inputs"]["input1"][0].get(obj, "무슨 쓰레기인지 잘 모르겠어. 주위를 정리하고 사진을 다시 찍어줄래?") for obj in detected_objects)
    else:
        predictions_text = "무슨 쓰레기인지 잘 모르겠어. 주위를 정리하고 사진을 다시 찍어줄래?"
    
    return fig, predictions_text

result_output = gr.Textbox(label="Result")

demo = gr.Interface(
    fn=predict_image, 
    inputs=gr.Image(type='pil'), 
    outputs=[gr.Plot(), result_output]
)
demo.launch(share=True)


c:\Users\chch4\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


glass: 93.30%
plastic: 6.65%
trash: 0.02%
paper: 0.02%
metal: 0.00%
